In [13]:
#if not already installed, install saxonche (https://www.saxonica.com/saxon-c/index.xml ) and xmltodict (https://github.com/martinblech/xmltodict/ ) libraries
!pip install saxonche
!pip install xmltodict

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import saxonche as saxon
import xmltodict, json

In [40]:
input_file_base = "example_files/HoffmannPlantsV3"
input_file = input_file_base + ".xml"
stylesheet_file = "abcd2bioschemas-xml.xslt"

In [16]:
from saxonche import PySaxonProcessor
with PySaxonProcessor(license=False) as proc:
    print("Test SaxonC on Python")
    print(proc.version)
    xslt30proc = proc.new_xslt30_processor()

Test SaxonC on Python
SaxonC-HE 12.4 from Saxonica


In [41]:
# transform ABCD XML to Bioschemas.org-XML using XSLT file 

#xslt30proc.transform_to_file(source_file="example_files/HoffmannPlantsV3.xml", stylesheet_file="abcd2bioschemas-xml.xslt", output_file="example_files/HoffmannPlantsV3_schema.xml")
schema_xml = xslt30proc.transform_to_string(source_file=input_file, stylesheet_file=stylesheet_file)
schema_xml

'<?xml version="1.0" encoding="UTF-8"?>\n<jsonld xmlns:abcd="http://www.tdwg.org/schemas/abcd/2.06"\n        xmlns:efg="http://www.synthesys.info/ABCDEFG/1.0"\n        xmlns:xs="http://www.w3.org/2001/XMLSchema"\n        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"\n        context="http://schema.org/"\n        type="Dataset"\n        id="https://data.bgbm.org/dataset/gfbio/0004/">\n   <id>https://data.bgbm.org/dataset/gfbio/0004/</id>\n   <name>Plants at Crater Road of Queen Elizabeth National Park, Uganda.</name>\n   <url>https://data.bgbm.org/dataset/gfbio/0004/</url>\n   <description>Plant occurrences at Crater Road, Queen Elizabeth National Park, Uganda (April 1995 until March 1997). Project data from Dr. Anke Hoffmann, supplementary data of the dissertation: "Hoffmann, Anke (1999). Habitat use and population dynamics of small mammals in the grasslands of the Queen Elizabeth National Park, Uganda. Dissertation Technische Universität Carolo-Wilhelmina zu Braunschweig. Brau

In [51]:
#transform XML structure to Python dictionary structure
schema_dict = xmltodict.parse(schema_xml)
schema_dict['jsonld']

OrderedDict([('@xmlns:abcd', 'http://www.tdwg.org/schemas/abcd/2.06'),
             ('@xmlns:efg', 'http://www.synthesys.info/ABCDEFG/1.0'),
             ('@xmlns:xs', 'http://www.w3.org/2001/XMLSchema'),
             ('@xmlns:xsi', 'http://www.w3.org/2001/XMLSchema-instance'),
             ('@context', 'http://schema.org/'),
             ('@type', 'Dataset'),
             ('@id', 'https://data.bgbm.org/dataset/gfbio/0004/'),
             ('id', 'https://data.bgbm.org/dataset/gfbio/0004/'),
             ('name',
              'Plants at Crater Road of Queen Elizabeth National Park, Uganda.'),
             ('url', 'https://data.bgbm.org/dataset/gfbio/0004/'),
             ('description',
              'Plant occurrences at Crater Road, Queen Elizabeth National Park, Uganda (April 1995 until March 1997). Project data from Dr. Anke Hoffmann, supplementary data of the dissertation: "Hoffmann, Anke (1999). Habitat use and population dynamics of small mammals in the grasslands of the Queen E

In [ ]:
#TODO: aggregate taxon names to higher level here 


In [52]:
#convert data values
#TODO: write algorithm to do that generically
try:
    schema_dict['jsonld']['isAccessibleForFree'] = bool(schema_dict['jsonld']['isAccessibleForFree'])
except KeyError:
    pass
    
try:
    schema_dict['jsonld']['size']['value'] = int(schema_dict['jsonld']['size']['value']['#text'])
except KeyError:
    pass
    
try:
    schema_dict['jsonld']['geo']['latitude'] = float(schema_dict['jsonld']['geo']['latitude']['#text'])
except KeyError:
    pass
    
try:
    schema_dict['jsonld']['geo']['longitude'] = float(schema_dict['jsonld']['geo']['longitude']['#text'])
except KeyError:
    pass

In [53]:
#export python 
schema_json = json.dumps(schema_dict['jsonld']) 

f = open(input_file_base+".json", "w")
f.write(schema_json)
f.close()